In [1]:
import langchain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.agents import load_tools, initialize_agent
import openai
import os
import wandb
from wandb.integration.langchain import WandbTracer
import json

In [14]:
wandb.init(
    project="generate_simple_prd",
    config={
        "model": "gpt-3.5-turbo",
        "temperature": 0
    },
    entity="arihantsheth",
    name="generate_simple_prd_v2",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arihantsheth. Use `wandb login --relogin` to force relogin


In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt_template = load_prompt("../prompt_templates/generate_prd_template_v2.json")

c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ariha\Desktop\Synap Labs\synap_labs_env\lib\site-packages\langchain\llms\openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
prompt_input_variables = {
    "wandb": dict(
        new_feature="Model Explainability Toolkit (MXT)",
        new_feature_desc="Enables users to interpret and explain the outputs and decision-making of their AI Models."
    ),
    "langchain": dict(
        new_feature="Model Training Wizard (MTW)",
        new_feature_desc=" A user-friendly interface that guides users through the process of training their own custom language models. It would provide step-by-step instructions, dataset management tools, and performance evaluation metrics to help users create high-quality models tailored to their specific needs."
    )
}

In [4]:
string_template = """\
You are a talkative product manager.

Now, you want to add the following new feature to this:
{new_feature}
Feature description: {new_feature_desc}

Create a Product Requirement Document (PRD) to add this new feature in the following format. Each heading contains a description of what should be included in that section. Return your output in a markdown format to maintain consistency:

1. What Are We Solving For?
    Give the high-level reasoning for this proposal. What is the problem or opportunity we’re pursuing? Who is this for? What segments or personas are impacted?

2. Why Does It Matter?
    Why is this initiative big enough to matter to Customers? What data or studies do we have to support this? 

3. What Does Success Look Like?
    What is the primary metric you’ll use to gauge success of this initiative? What is the target lift/impact to this metric, over what time period?
    What are the secondary metric(s) of success?

4. Background
    Include any necessary background research here, including data analysis, industry research, competitor analysis, quantitative or qualitative customer insights, etc.

5. Known Constraints
    Where applicable, lay out any constraints that must be taken into account when designing a solution. These constraints may related to usability (i.e. accessibility), time (seasonality), technology (platform capabilities or site performance), legal, and business guardrails. 

6. Solution Proposal
    Given the opportunity, the background, and the constraints, what are we proposing? List bullet points or insert sketches that describe the solution at a high level. If this initiative is successful, what will get better for the customers?

7. Risks & Assumptions
    What must we assume to be true in order for this initiative to be successful? What developments could cause this initiative to be unsuccessful? Risks include fraud/abuse potential, security…

8. Target Customers
>   8.1 Market
        Detail the specific regions & countries which the Product Development Team needs to build a solution for? Where we can lean into opportunities? Where do we need to mitigate emergent risks?
>   8.2 Audience Segments
        List the unique customer audiences this solution needs to address uniquely. 

9. User Experience
>   9.1 User Flows
        What screens, touch points or experiences that are being considered as part of this initiative? There may be several - and these may cross boundaries of products, experiences and user states. 

10. High Level Use Cases: Fill it in the given format

|    	| Name                                                       	| Description                                                                                                                                            	| PDT or Squad                                                                           	| Dependency?                                                                 	| Priority                                    	| Open Questions,  Notes & Discussions                                             	|
|----	|------------------------------------------------------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------	|----------------------------------------------------------------------------------------	|-----------------------------------------------------------------------------	|---------------------------------------------	|----------------------------------------------------------------------------------	|
| Desc  | Give this use case a name. e.g. Order Lifecycle Management 	| Note: this is not a user story. User stories are much more detailed/granular views of use cases. They represent the requirements & definition of done. 	| Name the Product Development team or squad if you know who will work on this use case. 	| If this use case has a dependency, list the use case # on which it depends. 	| MUST HAVE SHOULD HAVE COULD HAVE WON'T HAVE 	| Important things team members should know / things you're working to figure out. 	|

11. Rollout Approach
    How will we execute this rollout? Big-bang or gradual scale-up? A/B test or launch-and-measure? 
    Will the rollout happen in every market at once, or will markets ramp up one by one?
    Be inclusive of other key Product Development Team members in having this discussion, in particular Data Analysts, Data Science and Business Ops, along with Developers.
    Provide a comprehensive view into the Pros/Cons, what can/cannot be answered as a result, and reasoning behind the decision being made for the rollout approach. 

12. Measuring Success
    If A/B testing, describe the variants being tested, traffic allocations to each variant, and the KPIs being measured in the experiment. 
    Describe the desired cadence of performance readouts and align with Analytics to ensure it’s covered.
    As with the Rollout Approach section, please be sure to socialize this plan with your partners in Analytics, Data Science and Business Operations."""

In [11]:
# prompt_template = PromptTemplate(template=string_template, input_variables=["new_feature", "new_feature_desc"])

# prompt_template.save("../prompt_templates/generate_prd_template_v2.json")

In [6]:
prompts = dict()
for company in prompt_input_variables:
    prompts[company] = prompt_template.format(**prompt_input_variables[company])

In [10]:
with open("../company_prompts/prompt_inputs_v2.json", "w") as f:
    json.dump(prompt_input_variables, f)

with open("../company_prompts/prompts_v2.json", "w") as f:
    json.dump(prompts, f)

In [7]:
outputs = dict()

for company in prompts:
    print(f"Generating PRD for {company}...")
    prompt = prompts[company]
    
    try:
        # output = llm(prompt, callbacks=[WandbTracer()])
        output = llm(prompt)
    except Exception as e:
        print(e)
        output = f"ERROR: {e}"

    outputs[company] = output
    # wandb.log({f"{company}_prd": output})

Generating PRD for wandb...
Generating PRD for langchain...


In [8]:
from IPython.display import display, Markdown

for company in outputs:
    print(f"PRD for {company}:")
    display(Markdown(outputs[company]))
    break

PRD for wandb:


# Model Explainability Toolkit (MXT) Feature PRD

## 1. What Are We Solving For?
We are proposing to add a new feature to the Model Explainability Toolkit (MXT) that enables users to interpret and explain the outputs and decision-making of their AI Models. This feature aims to address the problem of lack of transparency and interpretability in AI models, which can hinder user trust and adoption. The target audience for this feature includes data scientists, machine learning engineers, and AI practitioners who want to gain insights into how their models make decisions.

## 2. Why Does It Matter?
This initiative is significant because it empowers customers to understand and explain the outputs and decision-making of their AI models. By providing interpretability and explainability, users can gain insights into the factors influencing their models' predictions, identify biases or errors, and build trust with stakeholders. Studies have shown that interpretability is crucial for regulatory compliance, ethical considerations, and user acceptance of AI models.

## 3. What Does Success Look Like?
The primary metric to gauge the success of this initiative is the increase in user satisfaction with model interpretability. We aim to achieve a target lift of 20% in user satisfaction within 6 months of launching the feature. The secondary metrics of success include an increase in user engagement with the MXT feature, a decrease in user-reported issues related to model interpretability, and positive feedback from users regarding the usefulness of the feature.

## 4. Background
Extensive research and industry analysis have highlighted the importance of model interpretability and explainability. Many AI models, such as deep neural networks, are often considered black boxes due to their complex architectures and lack of transparency. This can lead to challenges in understanding how models arrive at their predictions, making it difficult to trust and validate their outputs. Competitors in the market have started offering similar explainability features, and customer feedback indicates a strong demand for interpretability tools.

## 5. Known Constraints
- Usability: The solution should be designed to be user-friendly and accessible to users with varying levels of technical expertise.
- Time: The solution should be developed and launched within 6 months to meet market demand and stay ahead of competitors.
- Technology: The solution should leverage existing platform capabilities and ensure optimal site performance.
- Legal: The solution should comply with relevant data privacy and security regulations.
- Business: The solution should align with the company's strategic goals and financial constraints.

## 6. Solution Proposal
- Develop an intuitive user interface within the MXT platform that allows users to visualize and explore the decision-making process of their AI models.
- Provide feature importance rankings, highlighting the most influential factors in the model's predictions.
- Enable users to drill down into individual predictions to understand the specific factors that contributed to each decision.
- Offer model-agnostic interpretability techniques, such as LIME and SHAP, to accommodate a wide range of AI models.
- Provide explanations in both textual and graphical formats to cater to different user preferences.
- Improve the documentation and user guides to educate users on the importance of model interpretability and how to effectively use the MXT feature.

If this initiative is successful, customers will have a better understanding of their AI models' decision-making process, leading to increased trust, improved model performance, and enhanced compliance with regulatory requirements.

## 7. Risks & Assumptions
Risks:
- The interpretability techniques may not be applicable to all types of AI models, requiring additional research and development efforts.
- The explanations provided by the MXT feature may not always align with users' expectations, leading to potential user dissatisfaction.
- Ensuring the security and privacy of the model's internal workings and data used for explanations is crucial to mitigate potential risks of unauthorized access or misuse.

Assumptions:
- Users have a basic understanding of machine learning concepts and are willing to invest time in interpreting and explaining their models.
- The MXT feature will be well-received by users and positively impact their workflow and decision-making processes.

## 8. Target Customers
### 8.1 Market
The Product Development Team needs to build a solution for the global market, with a focus on regions and countries where AI adoption is high, such as the United States, Europe, and Asia. Opportunities can be leveraged in emerging markets where AI adoption is growing rapidly. Risks related to regulatory compliance and data privacy should be mitigated by ensuring the solution aligns with local laws and regulations.

### 8.2 Audience Segments
- Data scientists and machine learning engineers who develop and deploy AI models.
- AI practitioners who need to explain and justify the decisions made by their models to stakeholders.
- Compliance officers and auditors who require transparency and interpretability in AI models for regulatory purposes.

## 9. User Experience
### 9.1 User Flows
- User logs into the MXT platform and selects the AI model they want to interpret.
- User navigates to the "Explainability" section and chooses the desired interpretability technique.
- User explores the feature importance rankings and drills down into individual predictions for detailed explanations.
- User can customize the visualization and export the explanations for reporting or further analysis.
- User can access comprehensive documentation and tutorials to learn how to effectively interpret and explain their models.

## 10. High Level Use Cases:

|    	| Name                                                       	| Description                                                                                                                                            	| PDT or Squad                                                                           	| Dependency?                                                                 	| Priority                                    	| Open Questions,  Notes & Discussions                                             	|
|----	|------------------------------------------------------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------	|----------------------------------------------------------------------------------------	|-----------------------------------------------------------------------------	|---------------------------------------------	|----------------------------------------------------------------------------------	|
| 1 	| Feature Importance Ranking                                 	| Provide users with a ranked list of the most important features influencing the model's predictions.                                                 	| MXT Development Team                                                                    	| -                                                                           	| MUST HAVE                                    	| -                                                                                	|
| 2 	| Individual Prediction Explanation                          	| Enable users to explore the factors contributing to individual predictions made by the model.                                                        	| MXT Development Team                                                                    	| -                                                                           	| MUST HAVE                                    	| -                                                                                	|
| 3 	| Model-Agnostic Interpretability Techniques                 	| Implement popular interpretability techniques like LIME and SHAP that can be applied to a wide range of AI models.                                   	| MXT Development Team                                                                    	| -                                                                           	| SHOULD HAVE                                  	| -                                                                                	|
| 4 	| Customizable Visualization                                 	| Allow users to customize the visualization of explanations based on their preferences and reporting requirements.                                    	| MXT Development Team                                                                    	| -                                                                           	| SHOULD HAVE                                  	| -                                                                                	|
| 5 	| Documentation and Tutorials                                	| Improve the documentation and user guides to educate users on the importance of model interpretability and how to effectively use the MXT feature.   	| MXT Development Team                                                                    	| -                                                                           	| SHOULD HAVE                                  	| -                                                                                	|

## 11. Rollout Approach
The rollout approach for this feature will involve a gradual scale-up. We will initially launch the feature as a beta version to a select group of users, including existing customers who have expressed interest in interpretability tools. This will allow us to gather feedback, identify any potential issues, and make necessary improvements before a full-scale launch. The rollout will happen in multiple markets simultaneously to ensure a global reach and maximize user adoption. A/B testing will be conducted to measure the impact of the feature on user satisfaction and engagement.

## 12. Measuring Success
A/B testing will be conducted to compare the performance of the MXT feature with and without the new interpretability capabilities. The variants being tested include the control group (without the feature) and the experimental group (with the feature). Traffic allocation will be evenly split between the two groups. The primary KPIs being measured in the experiment include user satisfaction, user engagement, and user-reported issues related to model interpretability. Performance readouts will be conducted on a monthly basis, and the results will be shared with the Analytics, Data Science, and Business Operations teams for further analysis and insights.

In [9]:
for company in outputs:
    with open(f"../generated_prds/{company}_prd_v2.md", "w") as f:
        f.write(outputs[company])

In [28]:
wandb.finish()

langchain_prd,# Langchain Model Tr...
wandb_prd,# Product Requiremen...
